In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import random
import yfinance as yf
import datetime
import plotly.express as px
from aux_backtesting_modularizado import *

In [22]:
# 0. Definir estrategia
strategy = 'e_RSI'

# 1. Establecer fechas y lista de activos
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_tickers = list(sp500.Symbol)
sp500_tickers = [e.replace('.','-') for e in sp500_tickers]

# 2. Establecer reglas para comprar y vender, construir dataset con estrategia C, M, V y trades posibles
results = pd.DataFrame(columns=['metrica','grupo','valor'])
j = 0
while j < 3:
    j = j + 1
    print('Iteración', j)

    tickers = random.sample(sp500_tickers, 5)
    empieza = str(random.randint(2004, 2022)) + '-' + str(random.randint(1, 12)) + '-01'
    termina = datetime.datetime.strptime(empieza, "%Y-%m-%d") + datetime.timedelta(days=random.randint(360, 1000))
    
    print(tickers, empieza, termina)
    df_global, daily_returns = descarga(tickers, empieza, termina)
    df_global = build_individual_strategy(df_global, estrategia = strategy)    
    df_trades = build_trades(df_global, estrategia = strategy)

    # 3. Determinar reglas en caso de 0 o más de 1 activo disponible el mismo día, generar array de retornos
    benchmark_returns = yf.download(tickers, start=empieza, end=termina).iloc[:, 0:len(tickers)].pct_change().mean(axis=1)
    strategy_returns = build_strategy_returns(df_trades, daily_returns, benchmark_returns, duplicados='random', fr = 0)

    # 4. Análisis
    strategy_analysis = pd.DataFrame(index = benchmark_returns.index)
    strategy_analysis['benchmark'] = benchmark_returns
    strategy_analysis['strategy'] = strategy_returns
    sinteticos_px = (strategy_analysis+1).cumprod()
    res = resumen(sinteticos_px['strategy'], sinteticos_px['benchmark'])
    results = pd.concat([results, res])

Iteración 1
['WEC', 'DLTR', 'UNH', 'ROST', 'PYPL'] 2011-1-01 2012-05-02 00:00:00
[*********************100%***********************]  1 of 1 completed
1
[*********************100%***********************]  1 of 1 completed
2
[*********************100%***********************]  1 of 1 completed
3
[*********************100%***********************]  1 of 1 completed
4
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- PYPL: Data doesn't exist for startDate = 1293850800, endDate = 1335927600
5
[*********************100%***********************]  5 of 5 completed

1 Failed download:
- PYPL: Data doesn't exist for startDate = 1293850800, endDate = 1335927600
Iteración 2
['UHS', 'PTC', 'ZBH', 'BA', 'AOS'] 2015-4-01 2017-12-10 00:00:00
[*********************100%***********************]  1 of 1 completed
1
[*********************100%***********************]  1 of 1 completed
2
[*********************100%***********************]  1 of 1 completed
3
[************

In [23]:
fig = px.box(results, x="metrica", y="valor", color="grupo")
fig.update_traces(quartilemethod="exclusive")
fig.show()